# DESCARGA DE DATOS

In [1]:
#Create DatasetDict
from datasets import DatasetDict, Dataset, load_dataset
dataset_path = 'RikoteMaster/isear_augmented'
dataset_dict = load_dataset(dataset_path)

dataset_dict = dataset_dict.remove_columns('Augmented')
dataset_dict 


/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
Found cached dataset parquet (/root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Emotion', 'Text_processed'],
        num_rows: 10751
    })
})

In [2]:
#charge the train datasetDict as a df
df = dataset_dict['train'].to_pandas()
df.head()
#create id2label and label2id
id2label = {i: label for i, label in enumerate(df['Emotion'].unique())}
label2id = {label: i for i, label in enumerate(df['Emotion'].unique())}

#apply label2id to the datasetDict
dataset_dict = dataset_dict.map(lambda example: {'labels': label2id[example['Emotion']]}, remove_columns=['Emotion'])



Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-67b0a7624c7a5d2f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-161de27b49f6fdea.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-03e305454b1bb339.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-161de27b49f6fdea.arrow
Loading cached processed dataset at /roo

In [3]:
dataset_dict

DatasetDict({
    validation: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Text_processed', 'labels'],
        num_rows: 10751
    })
})

### Carga del tokenizador

In [4]:
from transformers import AutoTokenizer
model_ckpt = "bhadresh-savani/bert-base-go-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [5]:
def tokenize_text(examples):
    return tokenizer(examples["Text_processed"], padding="max_length")

In [6]:
dataset_dict = dataset_dict.map(tokenize_text, batched=True)
dataset_dict

Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-de0f2d38f00f62e0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d12b0dfd191548b2.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/RikoteMaster___parquet/RikoteMaster--isear_augmented-b6d7bc560c3e10d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c3255ff59bfc3ef6.arrow


DatasetDict({
    validation: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 752
    })
    test: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 752
    })
    train: Dataset({
        features: ['Text_processed', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10751
    })
})

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    
    return {'eval_accuracy': acc, 'f1': f1}

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True),

    }

device = 'cuda' if torch.cuda.is_available() else 'cpu'
def model_init(trial):
    if 'model' in locals():
        del model
        torch.cuda.empty_cache()
    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(id2label), ignore_mismatched_sizes=True).to(device)
    return model


def compute_objective(metrics):
    
    return metrics['eval_accuracy'] + metrics['f1']


batch_size = 16
epochs = 5

output_dir = './results_searching_hyperparameters'
logging_steps = len(dataset_dict['train']) // batch_size

args = TrainingArguments( 
                        output_dir=output_dir, 
                        num_train_epochs=epochs,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        evaluation_strategy='epoch',
                        logging_steps=logging_steps,
                        fp16=True,
                        push_to_hub=False,
                        # prevent saving checkpoints
                        save_strategy="steps",
                        save_steps=int(1e9),  # a large number
                    )

trainer = Trainer(
    model=None,
    args=args,
    train_dataset= dataset_dict['train'],
    eval_dataset= dataset_dict['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20
)

# Save best_trial to a text file

best_trial_dict = {
    'run_id': best_trial.run_id,
    'objective': best_trial.objective,
    'hyperparameters': best_trial.hyperparameters
}

# Save best_trial_dict to a text file
import json
with open('best_trial.txt', 'w') as file:
    file.write(json.dumps(best_trial_dict, indent=4))  # indent=4 for pretty printing




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 120
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda120_nocublaslt.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/python3.8/dist-packages/torch_tensorrt/lib'), PosixPath('/usr/local/cuda/compat/lib'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/lib/python3.8/dist-packages/torch/lib:/usr/local/lib/python3.8/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/usr/local/lib/python3.8/dist-

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.303600,1.300966,0.523936,0.529620
2,0.880200,1.397371,0.558511,0.545737
3,0.648400,1.249776,0.593085,0.593962
4,0.450400,1.343176,0.610372,0.614121
5,0.308500,1.522583,0.601064,0.600966


[I 2023-07-13 08:28:01,879] Trial 0 finished with value: 1.2020300378066537 and parameters: {'learning_rate': 4.717411969871788e-05, 'weight_decay': 0.0005941401383077225}. Best is trial 0 with value: 1.2020300378066537.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `n

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.801800,1.642926,0.396277,0.382355
2,1.568000,1.527882,0.453457,0.445599
3,1.463600,1.480834,0.473404,0.466023
4,1.393900,1.455724,0.476064,0.468261
5,1.370700,1.448826,0.482713,0.473731


...

[I 2023-07-13 11:07:35,883] Trial 17 finished with value: 1.220612041549753 and parameters: {'learning_rate': 2.7618727598129593e-05, 'weight_decay': 0.0011571098842707425}. Best is trial 11 with value: 1.234708469293003.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.385600,1.309354,0.538564,0.535635
2,0.985000,1.275773,0.550532,0.545244


[I 2023-07-13 11:13:18,318] Trial 18 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/bert-base-go-emotion and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.347600,1.310227,0.525266,0.529652
2,0.942600,1.332790,0.563830,0.555967
3,0.700600,1.276482,0.579787,0.581897


[I 2023-07-13 11:21:50,760] Trial 19 pruned. 


In [8]:
!nvidia-smi

Thu Jul 13 11:29:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN V      Off  | 00000000:01:00.0 Off |                  N/A |
| 34%   45C    P8    26W / 250W |  10004MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(id2label), ignore_mismatched_sizes=True)
#freeze all the parameters except the last layer, be sure that you freeze all excepting the last layer
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
from transformers import EarlyStoppingCallback
batch_size = 32
epochs = 30

output_dir = './results_freezed'
logging_steps = len(dataset_dict['train']) // batch_size
args = TrainingArguments( output_dir=output_dir, 
                        num_train_epochs=epochs,
                        learning_rate=2e-3,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        weight_decay=0.01,
                        evaluation_strategy='epoch',
                        save_strategy='epoch',
                        logging_steps=logging_steps,
                        fp16=True,
                        push_to_hub=false,
                        load_best_model_at_end=True,
                        metric_for_best_model='accuracy')

In [ ]:
trainer = Trainer(model=model,
                  args=args,
                  train_dataset=dataset_dict['train'],
                  eval_dataset=dataset_dict['validation'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience=int(0.2*epochs))])

In [ ]:
trainer.train()

In [ ]:
#push to hub
trainer.push_to_hub()

In [ ]:
#predict on test set
preds = trainer.predict(dataset_dict['test'])
preds = preds.predictions.argmax(-1)
#calculate accuracy
acc = accuracy_score(test_df['Emotion'], preds)

print(acc)

In [ ]:
for param in model.bert.encoder.layer[-2:].parameters():
    param.requires_grad = False


In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()